In [1]:
from pathlib import Path
import pandas as pd
import xlwings as xw
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from rdp import rdp

In [2]:
#Access input folder
input_dir1 = Path ("G:/Chamodi/LSDYNA_1D/Incident_pressure_time_history/Near_field_time_history_dataset_extra")
print ("1",input_dir1)

#Access folders inside input folder
input_dir2 =  [folder for folder in input_dir1.iterdir() if folder.is_dir()] # L13_3
print ("2",input_dir2)

# Define current work directory
current_dir = Path.cwd()

# Access the template to be used to generate files
excel_template = current_dir/ "Template.xlsx"

1 G:\Chamodi\LSDYNA_1D\Incident_pressure_time_history\Near_field_time_history_dataset_extra
2 [WindowsPath('G:/Chamodi/LSDYNA_1D/Incident_pressure_time_history/Near_field_time_history_dataset_extra/L13_11'), WindowsPath('G:/Chamodi/LSDYNA_1D/Incident_pressure_time_history/Near_field_time_history_dataset_extra/L13_15'), WindowsPath('G:/Chamodi/LSDYNA_1D/Incident_pressure_time_history/Near_field_time_history_dataset_extra/L13_19'), WindowsPath('G:/Chamodi/LSDYNA_1D/Incident_pressure_time_history/Near_field_time_history_dataset_extra/L13_23'), WindowsPath('G:/Chamodi/LSDYNA_1D/Incident_pressure_time_history/Near_field_time_history_dataset_extra/L13_27'), WindowsPath('G:/Chamodi/LSDYNA_1D/Incident_pressure_time_history/Near_field_time_history_dataset_extra/L13_3'), WindowsPath('G:/Chamodi/LSDYNA_1D/Incident_pressure_time_history/Near_field_time_history_dataset_extra/L13_31'), WindowsPath('G:/Chamodi/LSDYNA_1D/Incident_pressure_time_history/Near_field_time_history_dataset_extra/L13_35'), Wi

In [3]:
df_all = pd.DataFrame(columns=["Mass", "Standoff distance", "Arrival time", "AP", "Change time", "CP", "End time", "EP"])

for folder1 in input_dir2: # folder1-L13_1
    
    # EM
    parts = folder1.name.split('_')
    EM = round(int(parts[1])/2, 1)
    
    # Make a list of data file names
    excel_files = [file for file in folder1.rglob("*.xlsx") if file.stem.isdigit() and int(file.stem) % 2 != 0]
    
    # Access each file 
    for file1 in excel_files: # file1-1
        
        # Assign value to A based on file.stem
        file_name = file1.stem
        SD_mapping = {"1": 1, "3": 1.25, "5":1.5, "7":1.75, "9":2,
                      "11":2.25, "13":2.5, "15":2.75, "17":3}
        SD = SD_mapping.get(file_name, "Unknown")
        
        # Read data from Excel file
        df1 = pd.read_excel(file1)
        
        # Extract time and pressure columns
        time = df1["Time"].dropna()
        pressure = df1["Pressure"].dropna()
        
        # ARRIVAL
        AT = time[0]
        AP = pressure[0]
        
        for i in range(len(pressure) - 1, -1, -1):
            if pressure[i] < -1:
                end_index = i
                break
        # END
        ET = time[end_index]
        EP = pressure[end_index]
        
        pressure = pressure[:end_index + 1]
        time = time[:end_index + 1]
        
        zero_crossing_index = next((i for i, val in enumerate(pressure[:], start=0) if val <= 0), None)
        # CHANGE
        CT = time[zero_crossing_index]
        CP = pressure[zero_crossing_index] 

        # Creating a new DataFrame to append
        row_all = pd.DataFrame({"Mass": [EM], "Standoff distance": [SD],
                                "Arrival time": [AT],"AP": [AP],
                                "Change time": [CT], "CP": [CP],
                                "End time": [ET], "EP": [EP],
                                "Reduced":[len(pressure)]})
        df_all = pd.concat([df_all, row_all], ignore_index=True)
                        

In [4]:
# Write dataframes in excel template
all_path = 'IP_near_field_all_reduced2.xlsx'

df_all.to_excel(all_path, index=False)